In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

# Load and preprocess the data (same as before)
data = pd.read_csv('resampled_dataset.csv')
X = data.iloc[:, :-1].values / 255.0
y = data.iloc[:, -1].values

# Encode the target variable (same as before)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split the data (same as before)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Create an intermediate complexity MLP model
model = Sequential([
    Dense(256, activation='relu', input_shape=(9,), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(182, activation='softmax')
])

# Compile the model (same as before)
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for training (same as before)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=32,
    validation_split=0.2,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")

c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.0084 - loss: 5.7170 - val_accuracy: 0.0137 - val_loss: 5.4412 - learning_rate: 0.0010
Epoch 2/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0217 - loss: 5.2823 - val_accuracy: 0.0092 - val_loss: 5.3921 - learning_rate: 0.0010
Epoch 3/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0351 - loss: 4.9881 - val_accuracy: 0.0137 - val_loss: 5.3021 - learning_rate: 0.0010
Epoch 4/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0619 - loss: 4.6959 - val_accuracy: 0.0183 - val_loss: 5.1887 - learning_rate: 0.0010
Epoch 5/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0719 - loss: 4.4696 - val_accuracy: 0.0160 - val_loss: 5.0315 - learning_rate: 0.0010
Epoch 6/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0859 - loss: 4.3408 - val_accuracy: 0.0206 - val_loss: 4.8423 - learning_rate: 0.0010
Epoch 7/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1037 - loss: 4.1385 - val_ac

### Explain parameters for the code above and why i use these values : 

* Normalize the input data - Κανονικοποιήση
RGB values are typically in the range 0-255. Normalizing them to 0-1 can help the model learn more effectively. Πιο συγκεκριμένα η κανονικοποίηση μπορεί να βοηθήσει το μοντέλο να μάθει πιο γρήγορα, καθώς οι αλγόριθμοι βελτιστοποίησης (όπως η gradient descent) λειτουργούν πιο αποτελεσματικά όταν τα δεδομένα είναι κανονικοποιημένα.H κανονικοποίηση μπορεί να οδηγήσει σε καλύτερη απόδοση του μοντέλου, καθώς βοηθά στην αποφυγή προβλημάτων όπως το "vanishing gradient" (εξαφάνιση κλίσης) ή το "exploding gradient" (έκρηξη κλίσης).Πολλά νευρωνικά δίκτυα χρησιμοποιούν συναρτήσεις ενεργοποίησης όπως η SIGMOID ή tanh, οι οποίες έχουν σχεδιαστεί για να λειτουργούν καλύτερα με εισόδους μεταξύ 0 και 1.

- Encode the target value - Κωδικοποιήση 
Machine learning models, especially neural networks, work with numbers, not text. Encoding transforms these text labels into a numerical format the model can understand and process. For multi-class problems, encoded labels allow the use of appropriate loss functions like categorical cross-entropy.

- MLP Model overall structure 
1. Activation Functions:
- ReLU: Makes the model non-linear, allowing it to learn complex patterns. It's fast and helps prevent some training issues.
- Softmax: Turns the final layer's output into probabilities, so you can see which emotion the model thinks is most likely.

2. Regularization Techniques:

- L2 Regularization (0.01): Discourages the model from relying too heavily on any single feature. It's like telling the model "don't put all your eggs in one basket".
- Dropout (0.4): Randomly ignores 40% of neurons during training. This is like forcing the model to "think" in different ways, making it more robust.
- BatchNormalization: Keeps the data flowing through the network at a consistent scale, which can speed up learning.

3. Optimizer (Adam) and Learning Rate (0.001):

- Adam is like a smart teacher that adjusts how big of learning steps to take.
- 0.001 is a common starting point - not too fast, not too slow.


4. Loss Function (Categorical Cross-Entropy):

- This measures how wrong the model's predictions are. It's particularly good for problems with multiple classes like this one.


5. Training Parameters:

- Batch size (32): Processes 32 samples at a time. It's a balance between speed and memory use. This can herlp stabliize the learning process.
- Epochs (300): Maximum number of times to go through the entire dataset.
- Validation split (0.2): Uses 20% of data to check how well the model is learning.


6. Callbacks:

- ReduceLROnPlateau: If the model stops improving, it slows down the learning rate. Like being more careful when you're close to the right answer.
- EarlyStopping: Stops training if the model stops improving, saving time.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

# Load and preprocess the data (same as before)
data = pd.read_csv('x6perfect_reseampled.csv')
X = data.iloc[:, :-1].values / 255.0
y = data.iloc[:, -1].values

# Encode the target variable (same as before)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split the data (same as before)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Create an intermediate complexity MLP model
model = Sequential([
    Dense(256, activation='relu', input_shape=(9,), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(182, activation='softmax')
])

# Compile the model (same as before)
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for training (same as before)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=32,
    validation_split=0.2,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")